In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from math import sqrt

from sklearn.ensemble import RandomForestRegressor

In [2]:
data = pd.read_csv('Resources/Clean_Data/Final_Data.csv')
data.head()

,Zip_Code,Date,Date_Code,Year,Month,Sale_Price,Interest_Rate,Property_Tax,Rent_Price,Household_Income,...,FTE_Employed,Unemployed,Expense_Index,Average_Commute,Crime_Index,Loan_Amount,Loan_Term,Loan_R,Loan_Payment,Home_Affordability
0,32003,2019-01-01,201901,2019,1,244950.0,4.4640,3778,1113,94154.0,...,10671,1069,104,35,28,195960.0,360.0,1.003720,988.713278,0.126012
1,32003,2019-02-01,201902,2019,2,270000.0,4.3700,3778,1113,94154.0,...,10671,1069,104,35,28,216000.0,360.0,1.003642,1077.819242,0.137369
2,32003,2019-03-01,201903,2019,3,275000.0,4.2650,3778,1113,94154.0,...,10671,1069,104,35,28,220000.0,360.0,1.003554,1084.200547,0.138182
3,32003,2019-04-01,201904,2019,4,264500.0,4.1425,3778,1113,94154.0,...,10671,1069,104,35,28,211600.0,360.0,1.003452,1027.671397,0.130978
4,32003,2019-05-01,201905,2019,5,281000.0,4.0720,3778,1113,94154.0,...,10671,1069,104,35,28,224800.0,360.0,1.003393,1082.581718,0.137976


In [3]:
ml_data = data[['Zip_Code', 'Date_Code', 'Mobility_Rate', 'Expense_Index', 'Crime_Index' , 'Total_Vacant', 'Total_Dwellings', 'Total_Sales', 'FHA_Count', 'Home_Affordability', 'Rent_Affordability', 'Sale_Price' ]]
ml_data.head()

,Zip_Code,Date_Code,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price
0,32003,201901,0.182327,104,28,704,11724,32,7,0.126012,0.141853,244950.0
1,32003,201902,0.182327,104,28,704,11724,41,2,0.137369,0.141853,270000.0
2,32003,201903,0.182327,104,28,704,11724,21,7,0.138182,0.141853,275000.0
3,32003,201904,0.182327,104,28,704,11724,42,3,0.130978,0.141853,264500.0
4,32003,201905,0.182327,104,28,704,11724,66,8,0.137976,0.141853,281000.0


In [4]:
rf_data = ml_data.sort_values(by = ['Zip_Code','Date_Code'], ascending = [True, True])
rf_data['Zip_Code'] = rf_data['Zip_Code'].astype(str)
rf_data = rf_data.set_index('Zip_Code')

rf_data

,Date_Code,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price
Zip_Code,,,,,,,,,,,
32003,201901,0.182327,104,28,704,11724,32,7,0.126012,0.141853,244950.0
32003,201902,0.182327,104,28,704,11724,41,2,0.137369,0.141853,270000.0
32003,201903,0.182327,104,28,704,11724,21,7,0.138182,0.141853,275000.0
32003,201904,0.182327,104,28,704,11724,42,3,0.130978,0.141853,264500.0
32003,201905,0.182327,104,28,704,11724,66,8,0.137976,0.141853,281000.0
...,...,...,...,...,...,...,...,...,...,...,...
34997,202011,0.143471,91,113,2976,21551,132,11,0.173497,0.247189,259750.0
34997,202012,0.143471,91,113,2976,21551,142,16,0.196301,0.247189,297000.0
34997,202101,0.143471,91,113,2976,21551,81,10,0.202930,0.247189,305000.0


In [5]:
rf_data['Last_Month_Price'] = rf_data.groupby(['Zip_Code'])['Sale_Price'].shift()
rf_data['Last_2Month_Price'] = rf_data.groupby(['Zip_Code'])['Sale_Price'].shift(2)
rf_data['Last_3Month_Price'] = rf_data.groupby(['Zip_Code'])['Sale_Price'].shift(3)
# ml_data_3mo['Future_Month_Price'] = ml_data_3mo.groupby(['Zip_Code'])['Sale_Price'].shift(-1)
# ml_data_3mo['Future_2Month_Price'] = ml_data_3mo.groupby(['Zip_Code'])['Sale_Price'].shift(-2)
# ml_data_3mo['Future_3Month_Price'] = ml_data_3mo.groupby(['Zip_Code'])['Sale_Price'].shift(-3)


price_difference_1 = []
price_difference_2 = []
price_difference_3 = []


for column in rf_data[['Sale_Price', 'Last_Month_Price', 'Last_2Month_Price',
                    'Last_3Month_Price']]:
   # Select column contents by column name using [] operator
    current_sales = rf_data['Sale_Price'].values
    last_sales = rf_data['Last_Month_Price'].values
    last_2mo_sale = rf_data['Last_2Month_Price'].values
    last_3mo_sale = rf_data['Last_3Month_Price'].values
    price_difference_1.append(current_sales-last_sales)
    price_difference_2.append(last_sales-last_2mo_sale)
    price_difference_3.append(last_2mo_sale-last_3mo_sale)
    

rf_data['Last_Month_Diff'] = price_difference_1[0]
rf_data['Last_2Month_Diff'] = price_difference_2[0]
rf_data['Last_3Month_Diff'] = price_difference_3[0]

rf_data

,Date_Code,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Last_Month_Price,Last_2Month_Price,Last_3Month_Price,Last_Month_Diff,Last_2Month_Diff,Last_3Month_Diff
Zip_Code,,,,,,,,,,,,,,,,,
32003,201901,0.182327,104,28,704,11724,32,7,0.126012,0.141853,244950.0,NaN,NaN,NaN,NaN,NaN,NaN
32003,201902,0.182327,104,28,704,11724,41,2,0.137369,0.141853,270000.0,244950.0,NaN,NaN,25050.0,NaN,NaN
32003,201903,0.182327,104,28,704,11724,21,7,0.138182,0.141853,275000.0,270000.0,244950.0,NaN,5000.0,25050.0,NaN
32003,201904,0.182327,104,28,704,11724,42,3,0.130978,0.141853,264500.0,275000.0,270000.0,244950.0,-10500.0,5000.0,25050.0
32003,201905,0.182327,104,28,704,11724,66,8,0.137976,0.141853,281000.0,264500.0,275000.0,270000.0,16500.0,-10500.0,5000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34997,202011,0.143471,91,113,2976,21551,132,11,0.173497,0.247189,259750.0,285000.0,256000.0,251000.0,-25250.0,29000.0,5000.0
34997,202012,0.143471,91,113,2976,21551,142,16,0.196301,0.247189,297000.0,259750.0,285000.0,256000.0,37250.0,-25250.0,29000.0
34997,202101,0.143471,91,113,2976,21551,81,10,0.202930,0.247189,305000.0,297000.0,259750.0,285000.0,8000.0,37250.0,-25250.0


In [6]:
rf_data.replace([np.inf, -np.inf], np.nan, inplace=True)
rf_data.dropna(inplace=True)
rf_data.head()

,Date_Code,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Last_Month_Price,Last_2Month_Price,Last_3Month_Price,Last_Month_Diff,Last_2Month_Diff,Last_3Month_Diff
Zip_Code,,,,,,,,,,,,,,,,,
32003,201904,0.182327,104,28,704,11724,42,3,0.130978,0.141853,264500.0,275000.0,270000.0,244950.0,-10500.0,5000.0,25050.0
32003,201905,0.182327,104,28,704,11724,66,8,0.137976,0.141853,281000.0,264500.0,275000.0,270000.0,16500.0,-10500.0,5000.0
32003,201906,0.182327,104,28,704,11724,43,3,0.130690,0.141853,275000.0,281000.0,264500.0,275000.0,-6000.0,16500.0,-10500.0
32003,201907,0.182327,104,28,704,11724,90,7,0.133641,0.141853,282500.0,275000.0,281000.0,264500.0,7500.0,-6000.0,16500.0
32003,201908,0.182327,104,28,704,11724,64,5,0.125212,0.141853,269577.0,282500.0,275000.0,281000.0,-12923.0,7500.0,-6000.0


In [8]:
rf_data['Date_Code'] = rf_data['Date_Code'].astype(str)
rf_data

,Date_Code,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Last_Month_Price,Last_2Month_Price,Last_3Month_Price,Last_Month_Diff,Last_2Month_Diff,Last_3Month_Diff
Zip_Code,,,,,,,,,,,,,,,,,
32003,201904,0.182327,104,28,704,11724,42,3,0.130978,0.141853,264500.0,275000.0,270000.0,244950.0,-10500.0,5000.0,25050.0
32003,201905,0.182327,104,28,704,11724,66,8,0.137976,0.141853,281000.0,264500.0,275000.0,270000.0,16500.0,-10500.0,5000.0
32003,201906,0.182327,104,28,704,11724,43,3,0.130690,0.141853,275000.0,281000.0,264500.0,275000.0,-6000.0,16500.0,-10500.0
32003,201907,0.182327,104,28,704,11724,90,7,0.133641,0.141853,282500.0,275000.0,281000.0,264500.0,7500.0,-6000.0,16500.0
32003,201908,0.182327,104,28,704,11724,64,5,0.125212,0.141853,269577.0,282500.0,275000.0,281000.0,-12923.0,7500.0,-6000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34997,202011,0.143471,91,113,2976,21551,132,11,0.173497,0.247189,259750.0,285000.0,256000.0,251000.0,-25250.0,29000.0,5000.0
34997,202012,0.143471,91,113,2976,21551,142,16,0.196301,0.247189,297000.0,259750.0,285000.0,256000.0,37250.0,-25250.0,29000.0
34997,202101,0.143471,91,113,2976,21551,81,10,0.202930,0.247189,305000.0,297000.0,259750.0,285000.0,8000.0,37250.0,-25250.0


In [23]:
test = rf_data[rf_data['Date_Code'].str.contains('2021')]
test

,Date_Code,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Last_Month_Price,Last_2Month_Price,Last_3Month_Price,Last_Month_Diff,Last_2Month_Diff,Last_3Month_Diff
Zip_Code,,,,,,,,,,,,,,,,,
32003,202101,0.182327,104,28,704,11724,39,6,0.124589,0.141853,299900.0,294000.0,336000.0,330000.0,5900.0,-42000.0,6000.0
32003,202102,0.182327,104,28,704,11724,30,8,0.140027,0.141853,333800.0,299900.0,294000.0,336000.0,33900.0,5900.0,-42000.0
32003,202103,0.182327,104,28,704,11724,70,4,0.147681,0.141853,339950.0,333800.0,299900.0,294000.0,6150.0,33900.0,5900.0
32008,202101,0.067037,82,118,964,3247,6,0,0.153378,0.285180,132000.0,169500.0,72500.0,107000.0,-37500.0,97000.0,-34500.0
32008,202102,0.067037,82,118,964,3247,3,1,0.062420,0.285180,53200.0,132000.0,169500.0,72500.0,-78800.0,-37500.0,97000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34996,202102,0.141276,78,140,2780,9249,57,1,0.225364,0.209918,395000.0,345000.0,390000.0,400000.0,50000.0,-45000.0,-10000.0
34996,202103,0.141276,78,140,2780,9249,14,0,0.275304,0.209918,465950.0,395000.0,345000.0,390000.0,70950.0,50000.0,-45000.0
34997,202101,0.143471,91,113,2976,21551,81,10,0.202930,0.247189,305000.0,297000.0,259750.0,285000.0,8000.0,37250.0,-25250.0


In [10]:
sale_y_test = test['Sale_Price'].values

sale_X_test = test.drop(columns='Sale_Price')

sale_test_names = sale_X_test.columns

sale_X_test.head()

,Date_Code,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Last_Month_Price,Last_2Month_Price,Last_3Month_Price,Last_Month_Diff,Last_2Month_Diff,Last_3Month_Diff
Zip_Code,,,,,,,,,,,,,,,,
32003,202101,0.182327,104,28,704,11724,39,6,0.124589,0.141853,294000.0,336000.0,330000.0,5900.0,-42000.0,6000.0
32003,202102,0.182327,104,28,704,11724,30,8,0.140027,0.141853,299900.0,294000.0,336000.0,33900.0,5900.0,-42000.0
32003,202103,0.182327,104,28,704,11724,70,4,0.147681,0.141853,333800.0,299900.0,294000.0,6150.0,33900.0,5900.0
32008,202101,0.067037,82,118,964,3247,6,0,0.153378,0.285180,169500.0,72500.0,107000.0,-37500.0,97000.0,-34500.0
32008,202102,0.067037,82,118,964,3247,3,1,0.062420,0.285180,132000.0,169500.0,72500.0,-78800.0,-37500.0,97000.0


In [11]:
training = rf_data[~rf_data['Date_Code'].str.contains('2021')]
training

,Date_Code,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Last_Month_Price,Last_2Month_Price,Last_3Month_Price,Last_Month_Diff,Last_2Month_Diff,Last_3Month_Diff
Zip_Code,,,,,,,,,,,,,,,,,
32003,201904,0.182327,104,28,704,11724,42,3,0.130978,0.141853,264500.0,275000.0,270000.0,244950.0,-10500.0,5000.0,25050.0
32003,201905,0.182327,104,28,704,11724,66,8,0.137976,0.141853,281000.0,264500.0,275000.0,270000.0,16500.0,-10500.0,5000.0
32003,201906,0.182327,104,28,704,11724,43,3,0.130690,0.141853,275000.0,281000.0,264500.0,275000.0,-6000.0,16500.0,-10500.0
32003,201907,0.182327,104,28,704,11724,90,7,0.133641,0.141853,282500.0,275000.0,281000.0,264500.0,7500.0,-6000.0,16500.0
32003,201908,0.182327,104,28,704,11724,64,5,0.125212,0.141853,269577.0,282500.0,275000.0,281000.0,-12923.0,7500.0,-6000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34997,202008,0.143471,91,113,2976,21551,123,18,0.171370,0.247189,251000.0,255000.0,259900.0,240000.0,-4000.0,-4900.0,19900.0
34997,202009,0.143471,91,113,2976,21551,115,18,0.173776,0.247189,256000.0,251000.0,255000.0,259900.0,5000.0,-4000.0,-4900.0
34997,202010,0.143471,91,113,2976,21551,139,17,0.192070,0.247189,285000.0,256000.0,251000.0,255000.0,29000.0,5000.0,-4000.0


In [12]:
sale_y_train= training['Sale_Price'].values

sale_X_train = training.drop(columns='Sale_Price')

sale_train_names = sale_X_train.columns

sale_X_train.head()

,Date_Code,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Last_Month_Price,Last_2Month_Price,Last_3Month_Price,Last_Month_Diff,Last_2Month_Diff,Last_3Month_Diff
Zip_Code,,,,,,,,,,,,,,,,
32003,201904,0.182327,104,28,704,11724,42,3,0.130978,0.141853,275000.0,270000.0,244950.0,-10500.0,5000.0,25050.0
32003,201905,0.182327,104,28,704,11724,66,8,0.137976,0.141853,264500.0,275000.0,270000.0,16500.0,-10500.0,5000.0
32003,201906,0.182327,104,28,704,11724,43,3,0.130690,0.141853,281000.0,264500.0,275000.0,-6000.0,16500.0,-10500.0
32003,201907,0.182327,104,28,704,11724,90,7,0.133641,0.141853,275000.0,281000.0,264500.0,7500.0,-6000.0,16500.0
32003,201908,0.182327,104,28,704,11724,64,5,0.125212,0.141853,282500.0,275000.0,281000.0,-12923.0,7500.0,-6000.0


In [13]:
#Random Forest Regressor
rf_regr = RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=42)
rf_regr.fit(sale_X_train, sale_y_train)

RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=42)

In [14]:
print(f"Training Data Score: {rf_regr.score(sale_X_train, sale_y_train)}")
print(f"Testing Data Score: {rf_regr.score(sale_X_test, sale_y_test)}")

Training Data Score: 0.9981768655137256
Testing Data Score: 0.9781503842644864


In [15]:
rf_pred = rf_regr.predict(sale_X_test)

print('Mean Error = %.5f' % np.sqrt(mean_squared_log_error(sale_y_test, rf_pred)))

Mean Error = 0.07048


In [16]:
output_price = pd.DataFrame({'Sale_Price_Prediction':rf_pred, 'Actual': sale_y_test})
output_price

,Sale_Price_Prediction,Actual
0,300906.904,299900.0
1,333540.000,333800.0
2,338967.133,339950.0
3,133484.625,132000.0
4,67616.395,53200.0
...,...,...
2537,395200.821,395000.0
2538,464942.875,465950.0
2539,305423.011,305000.0
2540,300499.381,300000.0


In [17]:
# Calculate mean error for each row and add to new column
output_price['Sale_Price_RMSLE'] = (np.sqrt(mean_squared_log_error(output_price['Actual'], 
                                    output_price['Sale_Price_Prediction'])))

output_price = output_price.drop(columns='Actual')

output_price

,Sale_Price_Prediction,Sale_Price_RMSLE
0,300906.904,0.070478
1,333540.000,0.070478
2,338967.133,0.070478
3,133484.625,0.070478
4,67616.395,0.070478
...,...,...
2537,395200.821,0.070478
2538,464942.875,0.070478
2539,305423.011,0.070478
2540,300499.381,0.070478


In [18]:
full_output_data = test
full_output_data.reset_index(inplace=True)
output_price.reset_index(inplace=True)

price_final = pd.concat((full_output_data, output_price), axis=1)

price_final

,Zip_Code,Date_Code,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,...,Sale_Price,Last_Month_Price,Last_2Month_Price,Last_3Month_Price,Last_Month_Diff,Last_2Month_Diff,Last_3Month_Diff,index,Sale_Price_Prediction,Sale_Price_RMSLE
0,32003,202101,0.182327,104,28,704,11724,39,6,0.124589,...,299900.0,294000.0,336000.0,330000.0,5900.0,-42000.0,6000.0,0,300906.904,0.070478
1,32003,202102,0.182327,104,28,704,11724,30,8,0.140027,...,333800.0,299900.0,294000.0,336000.0,33900.0,5900.0,-42000.0,1,333540.000,0.070478
2,32003,202103,0.182327,104,28,704,11724,70,4,0.147681,...,339950.0,333800.0,299900.0,294000.0,6150.0,33900.0,5900.0,2,338967.133,0.070478
3,32008,202101,0.067037,82,118,964,3247,6,0,0.153378,...,132000.0,169500.0,72500.0,107000.0,-37500.0,97000.0,-34500.0,3,133484.625,0.070478
4,32008,202102,0.067037,82,118,964,3247,3,1,0.062420,...,53200.0,132000.0,169500.0,72500.0,-78800.0,-37500.0,97000.0,4,67616.395,0.070478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2537,34996,202102,0.141276,78,140,2780,9249,57,1,0.225364,...,395000.0,345000.0,390000.0,400000.0,50000.0,-45000.0,-10000.0,2537,395200.821,0.070478
2538,34996,202103,0.141276,78,140,2780,9249,14,0,0.275304,...,465950.0,395000.0,345000.0,390000.0,70950.0,50000.0,-45000.0,2538,464942.875,0.070478
2539,34997,202101,0.143471,91,113,2976,21551,81,10,0.202930,...,305000.0,297000.0,259750.0,285000.0,8000.0,37250.0,-25250.0,2539,305423.011,0.070478
2540,34997,202102,0.143471,91,113,2976,21551,116,7,0.201553,...,300000.0,305000.0,297000.0,259750.0,-5000.0,8000.0,37250.0,2540,300499.381,0.070478


## FHA Data

In [24]:
fha_y_test = test['FHA_Count'].values

fha_X_test = test.drop(columns='FHA_Count')

fha_test_names = fha_X_test.columns

fha_X_test.head()

,Date_Code,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,Home_Affordability,Rent_Affordability,Sale_Price,Last_Month_Price,Last_2Month_Price,Last_3Month_Price,Last_Month_Diff,Last_2Month_Diff,Last_3Month_Diff
Zip_Code,,,,,,,,,,,,,,,,
32003,202101,0.182327,104,28,704,11724,39,0.124589,0.141853,299900.0,294000.0,336000.0,330000.0,5900.0,-42000.0,6000.0
32003,202102,0.182327,104,28,704,11724,30,0.140027,0.141853,333800.0,299900.0,294000.0,336000.0,33900.0,5900.0,-42000.0
32003,202103,0.182327,104,28,704,11724,70,0.147681,0.141853,339950.0,333800.0,299900.0,294000.0,6150.0,33900.0,5900.0
32008,202101,0.067037,82,118,964,3247,6,0.153378,0.285180,132000.0,169500.0,72500.0,107000.0,-37500.0,97000.0,-34500.0
32008,202102,0.067037,82,118,964,3247,3,0.062420,0.285180,53200.0,132000.0,169500.0,72500.0,-78800.0,-37500.0,97000.0


In [25]:
fha_y_train= training['FHA_Count'].values

fha_X_train = training.drop(columns='FHA_Count')

fha_train_names = fha_X_train.columns

fha_X_train.head()

,Date_Code,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,Home_Affordability,Rent_Affordability,Sale_Price,Last_Month_Price,Last_2Month_Price,Last_3Month_Price,Last_Month_Diff,Last_2Month_Diff,Last_3Month_Diff
Zip_Code,,,,,,,,,,,,,,,,
32003,201904,0.182327,104,28,704,11724,42,0.130978,0.141853,264500.0,275000.0,270000.0,244950.0,-10500.0,5000.0,25050.0
32003,201905,0.182327,104,28,704,11724,66,0.137976,0.141853,281000.0,264500.0,275000.0,270000.0,16500.0,-10500.0,5000.0
32003,201906,0.182327,104,28,704,11724,43,0.130690,0.141853,275000.0,281000.0,264500.0,275000.0,-6000.0,16500.0,-10500.0
32003,201907,0.182327,104,28,704,11724,90,0.133641,0.141853,282500.0,275000.0,281000.0,264500.0,7500.0,-6000.0,16500.0
32003,201908,0.182327,104,28,704,11724,64,0.125212,0.141853,269577.0,282500.0,275000.0,281000.0,-12923.0,7500.0,-6000.0


In [26]:
rf_regr_2 = RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=42)
rf_regr_2.fit(fha_X_train, fha_y_train)

RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=42)

In [27]:
print(f"Training Data Score: {rf_regr_2.score(fha_X_train, fha_y_train)}")
print(f"Testing Data Score: {rf_regr_2.score(fha_X_test, fha_y_test)}")

Training Data Score: 0.9828581982586339
Testing Data Score: 0.8628764337929532


In [28]:
rf_pred_fha = rf_regr_2.predict(fha_X_test)

print('Mean Error = %.5f' % np.sqrt(mean_squared_log_error(fha_y_test, rf_pred_fha)))

Mean Error = 0.49515


In [29]:
output_fha = pd.DataFrame({'FHA_Count_Prediction':rf_pred_fha, 'Actual': fha_y_test})
output_fha

,FHA_Count_Prediction,Actual
0,6.639,6
1,6.177,8
2,7.632,4
3,1.148,0
4,1.107,1
...,...,...
2537,0.292,1
2538,0.235,0
2539,10.899,10
2540,12.718,7


In [30]:
# Calculate mean error for each row and add to new column
output_fha['FHA_Count_RMSLE'] = (np.sqrt(mean_squared_log_error(output_fha['Actual'], output_fha['FHA_Count_Prediction'])))

output_fha = output_fha.drop(columns='Actual')
output_fha
# count_pre = output_fha[output_fha['FHA_Count_Prediction']]

,FHA_Count_Prediction,FHA_Count_RMSLE
0,6.639,0.495148
1,6.177,0.495148
2,7.632,0.495148
3,1.148,0.495148
4,1.107,0.495148
...,...,...
2537,0.292,0.495148
2538,0.235,0.495148
2539,10.899,0.495148
2540,12.718,0.495148


In [31]:
output_fha.reset_index(inplace=True)

fha_final = pd.concat((full_output_data, output_fha), axis=1)

fha_final

,Zip_Code,Date_Code,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,...,Sale_Price,Last_Month_Price,Last_2Month_Price,Last_3Month_Price,Last_Month_Diff,Last_2Month_Diff,Last_3Month_Diff,index,FHA_Count_Prediction,FHA_Count_RMSLE
0,32003,202101,0.182327,104,28,704,11724,39,6,0.124589,...,299900.0,294000.0,336000.0,330000.0,5900.0,-42000.0,6000.0,0,6.639,0.495148
1,32003,202102,0.182327,104,28,704,11724,30,8,0.140027,...,333800.0,299900.0,294000.0,336000.0,33900.0,5900.0,-42000.0,1,6.177,0.495148
2,32003,202103,0.182327,104,28,704,11724,70,4,0.147681,...,339950.0,333800.0,299900.0,294000.0,6150.0,33900.0,5900.0,2,7.632,0.495148
3,32008,202101,0.067037,82,118,964,3247,6,0,0.153378,...,132000.0,169500.0,72500.0,107000.0,-37500.0,97000.0,-34500.0,3,1.148,0.495148
4,32008,202102,0.067037,82,118,964,3247,3,1,0.062420,...,53200.0,132000.0,169500.0,72500.0,-78800.0,-37500.0,97000.0,4,1.107,0.495148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2537,34996,202102,0.141276,78,140,2780,9249,57,1,0.225364,...,395000.0,345000.0,390000.0,400000.0,50000.0,-45000.0,-10000.0,2537,0.292,0.495148
2538,34996,202103,0.141276,78,140,2780,9249,14,0,0.275304,...,465950.0,395000.0,345000.0,390000.0,70950.0,50000.0,-45000.0,2538,0.235,0.495148
2539,34997,202101,0.143471,91,113,2976,21551,81,10,0.202930,...,305000.0,297000.0,259750.0,285000.0,8000.0,37250.0,-25250.0,2539,10.899,0.495148
2540,34997,202102,0.143471,91,113,2976,21551,116,7,0.201553,...,300000.0,305000.0,297000.0,259750.0,-5000.0,8000.0,37250.0,2540,12.718,0.495148


## Total Sales Predict

In [33]:
ts_y_test = test['Total_Sales'].values

ts_X_test = test.drop(columns='Total_Sales')

ts_test_names = ts_X_test.columns

ts_X_test.shape

(2542, 16)

In [34]:
ts_y_train= training['Total_Sales'].values

ts_X_train = training.drop(columns='Total_Sales')

ts_train_names = ts_X_train.columns

ts_X_train.head()

,Date_Code,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Last_Month_Price,Last_2Month_Price,Last_3Month_Price,Last_Month_Diff,Last_2Month_Diff,Last_3Month_Diff
Zip_Code,,,,,,,,,,,,,,,,
32003,201904,0.182327,104,28,704,11724,3,0.130978,0.141853,264500.0,275000.0,270000.0,244950.0,-10500.0,5000.0,25050.0
32003,201905,0.182327,104,28,704,11724,8,0.137976,0.141853,281000.0,264500.0,275000.0,270000.0,16500.0,-10500.0,5000.0
32003,201906,0.182327,104,28,704,11724,3,0.130690,0.141853,275000.0,281000.0,264500.0,275000.0,-6000.0,16500.0,-10500.0
32003,201907,0.182327,104,28,704,11724,7,0.133641,0.141853,282500.0,275000.0,281000.0,264500.0,7500.0,-6000.0,16500.0
32003,201908,0.182327,104,28,704,11724,5,0.125212,0.141853,269577.0,282500.0,275000.0,281000.0,-12923.0,7500.0,-6000.0


In [35]:
rf_regr_3 = RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=42)
rf_regr_3.fit(ts_X_train, ts_y_train)

RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=42)

In [36]:
print(f"Training Data Score: {rf_regr_3.score(ts_X_train, ts_y_train)}")
print(f"Testing Data Score: {rf_regr_3.score(ts_X_test, ts_y_test)}")

Training Data Score: 0.986467622176216
Testing Data Score: 0.631339745347925


In [37]:
rf_pred_ts = rf_regr_3.predict(ts_X_test)

print('Mean Error = %.5f' % np.sqrt(mean_squared_log_error(ts_y_test, rf_pred_ts)))

Mean Error = 0.51120


In [38]:
output_ts = pd.DataFrame({'Total_Sales_Predicted':rf_pred_ts, 'Actual': ts_y_test})
output_ts

,Total_Sales_Predicted,Actual
0,48.214,39
1,43.022,30
2,38.334,70
3,5.095,6
4,5.251,3
...,...,...
2537,53.668,57
2538,49.316,14
2539,126.088,81
2540,130.039,116


In [39]:
# Calculate mean error for each row and add to new column
output_ts['Total_Sales_Predicted_RMSLE'] = (np.sqrt(mean_squared_log_error(output_ts['Actual'], output_ts['Total_Sales_Predicted'])))

output_ts = output_ts.drop(columns='Actual')
output_ts

,Total_Sales_Predicted,Total_Sales_Predicted_RMSLE
0,48.214,0.511203
1,43.022,0.511203
2,38.334,0.511203
3,5.095,0.511203
4,5.251,0.511203
...,...,...
2537,53.668,0.511203
2538,49.316,0.511203
2539,126.088,0.511203
2540,130.039,0.511203


In [40]:
output_ts.reset_index(inplace=True)

ts_final = pd.concat((full_output_data, output_ts), axis=1)

ts_final

,Zip_Code,Date_Code,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,...,Sale_Price,Last_Month_Price,Last_2Month_Price,Last_3Month_Price,Last_Month_Diff,Last_2Month_Diff,Last_3Month_Diff,index,Total_Sales_Predicted,Total_Sales_Predicted_RMSLE
0,32003,202101,0.182327,104,28,704,11724,39,6,0.124589,...,299900.0,294000.0,336000.0,330000.0,5900.0,-42000.0,6000.0,0,48.214,0.511203
1,32003,202102,0.182327,104,28,704,11724,30,8,0.140027,...,333800.0,299900.0,294000.0,336000.0,33900.0,5900.0,-42000.0,1,43.022,0.511203
2,32003,202103,0.182327,104,28,704,11724,70,4,0.147681,...,339950.0,333800.0,299900.0,294000.0,6150.0,33900.0,5900.0,2,38.334,0.511203
3,32008,202101,0.067037,82,118,964,3247,6,0,0.153378,...,132000.0,169500.0,72500.0,107000.0,-37500.0,97000.0,-34500.0,3,5.095,0.511203
4,32008,202102,0.067037,82,118,964,3247,3,1,0.062420,...,53200.0,132000.0,169500.0,72500.0,-78800.0,-37500.0,97000.0,4,5.251,0.511203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2537,34996,202102,0.141276,78,140,2780,9249,57,1,0.225364,...,395000.0,345000.0,390000.0,400000.0,50000.0,-45000.0,-10000.0,2537,53.668,0.511203
2538,34996,202103,0.141276,78,140,2780,9249,14,0,0.275304,...,465950.0,395000.0,345000.0,390000.0,70950.0,50000.0,-45000.0,2538,49.316,0.511203
2539,34997,202101,0.143471,91,113,2976,21551,81,10,0.202930,...,305000.0,297000.0,259750.0,285000.0,8000.0,37250.0,-25250.0,2539,126.088,0.511203
2540,34997,202102,0.143471,91,113,2976,21551,116,7,0.201553,...,300000.0,305000.0,297000.0,259750.0,-5000.0,8000.0,37250.0,2540,130.039,0.511203


In [42]:
final_rf_data = pd.concat((full_output_data, output_price, output_fha, output_ts), axis=1)
final_rf_data

,Zip_Code,Date_Code,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,...,Last_3Month_Diff,index,Sale_Price_Prediction,Sale_Price_RMSLE,index,FHA_Count_Prediction,FHA_Count_RMSLE,index,Total_Sales_Predicted,Total_Sales_Predicted_RMSLE
0,32003,202101,0.182327,104,28,704,11724,39,6,0.124589,...,6000.0,0,300906.904,0.070478,0,6.639,0.495148,0,48.214,0.511203
1,32003,202102,0.182327,104,28,704,11724,30,8,0.140027,...,-42000.0,1,333540.000,0.070478,1,6.177,0.495148,1,43.022,0.511203
2,32003,202103,0.182327,104,28,704,11724,70,4,0.147681,...,5900.0,2,338967.133,0.070478,2,7.632,0.495148,2,38.334,0.511203
3,32008,202101,0.067037,82,118,964,3247,6,0,0.153378,...,-34500.0,3,133484.625,0.070478,3,1.148,0.495148,3,5.095,0.511203
4,32008,202102,0.067037,82,118,964,3247,3,1,0.062420,...,97000.0,4,67616.395,0.070478,4,1.107,0.495148,4,5.251,0.511203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2537,34996,202102,0.141276,78,140,2780,9249,57,1,0.225364,...,-10000.0,2537,395200.821,0.070478,2537,0.292,0.495148,2537,53.668,0.511203
2538,34996,202103,0.141276,78,140,2780,9249,14,0,0.275304,...,-45000.0,2538,464942.875,0.070478,2538,0.235,0.495148,2538,49.316,0.511203
2539,34997,202101,0.143471,91,113,2976,21551,81,10,0.202930,...,-25250.0,2539,305423.011,0.070478,2539,10.899,0.495148,2539,126.088,0.511203
2540,34997,202102,0.143471,91,113,2976,21551,116,7,0.201553,...,37250.0,2540,300499.381,0.070478,2540,12.718,0.495148,2540,130.039,0.511203


In [44]:
final_rf_data.to_csv('Resources/ML_Outputs/random_forest_model.csv', index=False)